In [1]:
import os
from llm_bases.chatglm6b import ChatGML6B
glm = ChatGML6B()

/root/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 8/8 [00:09<00:00,  1.22s/it]


In [2]:
prediction_layer = glm.condgen.lm_head.float().cuda()

In [3]:
import torch
hs = torch.normal(0, 1, [4096]).cuda()
h0 = torch.normal(0, 10, [4096]).cuda()
h1 = hs - h0

In [4]:
prediction = torch.argmax(prediction_layer(hs))
print("Expected prediction: ", prediction.item())

Expected prediction:  94781


In [5]:
try:
    import sys
    del sys.modules['split_llm.glm6b.secure_inference']
    del sys.modules['homomorphic_encryption.bfv']
    print("delete complete!")
except Exception as e:
    print(e)
from split_llm.glm6b.secure_inference import GLM_PredictionProtocol
from split_llm.common.torch_utils import relative_error

'split_llm.glm6b.secure_inference'


In [6]:
from split_llm.common.communication import Communication, Node, SimulatedCommunication
communication = SimulatedCommunication(["n0", "n1", "n2"])
communication.new_stage("Test")

n0 = Node(communication, "n0")
n1 = Node(communication, "n1")
n2 = Node(communication, "n2")

n0.space.final_dense = prediction_layer

prediction_protocol = GLM_PredictionProtocol(n0, n1, n2, 10, device="cuda")
prediction_protocol.prepare()
prediction_protocol.offline_execute()

In [7]:
n0.storage[f"{prediction_protocol.name}:x0"] = h0
n1.storage[f"{prediction_protocol.name}:x1"] = h1
prediction_protocol.online_execute()
print("Computed prediction: ", n1.storage[f"{prediction_protocol.name}:z"])

Computed prediction:  [94781]


In [10]:
prediction_layer.weight

Parameter containing:
tensor([[-8.4076e-03, -9.3689e-03, -5.4436e-03,  ..., -6.4545e-03,
          1.6998e-02,  1.1108e-02],
        [-1.0071e-02,  5.8022e-03,  4.8018e-04,  ..., -4.2701e-04,
          1.0252e-03, -1.6556e-03],
        [ 1.9424e-02,  6.3477e-03,  2.4933e-02,  ...,  5.7297e-03,
          1.2512e-02,  9.4147e-03],
        ...,
        [-1.0078e-02,  3.0041e-03,  2.4376e-03,  ..., -4.7684e-06,
          1.5430e-03,  1.1053e-03],
        [-9.9945e-03,  4.4479e-03,  6.2141e-03,  ...,  1.7560e-04,
          1.4286e-03, -1.1883e-03],
        [-9.3536e-03,  1.7376e-03,  5.7373e-03,  ..., -1.0910e-03,
          4.3945e-03, -1.2541e-03]], device='cuda:0')